Khai báo các thư viện cần thiết

In [3]:
import numpy as np
import cv2
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from skimage.transform import resize

Đọc dữ liệu

In [20]:
path = 'D:/Code/python/mat3533/practice08/data/face_data/'
ids = range(1, 16) # 15 people
states = ['centerlight', 'glasses', 'happy', 'leftlight',
'noglasses', 'normal', 'rightlight','sad',
'sleepy', 'surprised', 'wink' ]
prefix = 'subject'
surfix = '.png' #file extension is png
# open one picture to get the image's size
fn = prefix + '01.' + states[0] + surfix
im = cv2.imread(path + fn, 0)
h = im.shape[0] # hight
w = im.shape[1] # width
D = h * w
N = len(states)*15
print(N, D, h, w)
X = np.zeros((D, N))
# collect all data
count = 0
# there are 15 people
for person_id in range(1, 16):
    for state in states:
        # get name of each image file
        fn = path + prefix + str(person_id).zfill(2) + '.' + state + surfix
        # open the file and read as grey image
        tmp = cv2.imread(fn, cv2.IMREAD_GRAYSCALE)
        # then add image to dataset X
        X[:, count] = tmp.reshape(D)
        count += 1
y = np.repeat(range(1, 16), len(states))

165 77760 243 320


Phân chia dữ  liệu theo tỉ lệ train:test=0.8:0.2

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X.T, y, test_size=0.2, random_state=42)

In [23]:
import tensorflow as tf

y_train = tf.keras.utils.to_categorical(y_train, num_classes=16)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=16)

In [27]:
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense

model=tf.keras.Sequential()
model.add(tf.keras.layers.Reshape((243,320,1),input_shape=(77760,)))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(32, 
                                 kernel_size=(3,3), 
                                 activation='relu'))

model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

model.add(tf.keras.layers.Flatten())


model.add(tf.keras.layers.Dense(128, activation='relu'))


model.add(tf.keras.layers.Dropout(0.25))


model.add(tf.keras.layers.Dense(16, activation='softmax'))
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', metrics=['accuracy'])


In [28]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=16,
          batch_size=32)

Epoch 1/16
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.0487 - loss: 50.7819 - val_accuracy: 0.0606 - val_loss: 152.7256
Epoch 2/16
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.1965 - loss: 12.2430 - val_accuracy: 0.2727 - val_loss: 5.1503
Epoch 3/16
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.6033 - loss: 1.4740 - val_accuracy: 0.7879 - val_loss: 0.3568
Epoch 4/16
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.9707 - loss: 0.4044 - val_accuracy: 0.5758 - val_loss: 1.5881
Epoch 5/16
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 1.0000 - loss: 0.0709 - val_accuracy: 0.7879 - val_loss: 0.5447
Epoch 6/16
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 1.0000 - loss: 0.0260 - val_accuracy: 0.9091 - val_loss: 0.3487
Epoch 7/16
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 1.0000 - loss: 0.0041 - val_accuracy: 0.8788 - val_loss: 0.3482
Epoch 8/16
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 0.8788 - val_loss: 0.2508
Epoc

In [29]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis = 1)
y_pred

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step


array([13, 11, 12,  6,  9,  3, 15,  5, 10, 14,  2,  8,  2, 10,  3,  3, 13,
       10, 14,  2,  7,  2,  1,  1,  9, 15,  9,  6, 13, 15, 13,  8,  6],
      dtype=int64)

In [31]:
train_accuracy = tf.keras.metrics.Accuracy()
train_recall = tf.keras.metrics.Recall()
train_precision = tf.keras.metrics.Precision()

train_accuracy.update_state(tf.argmax(y_train, axis=1), tf.argmax(model.predict(X_train), axis=1))
train_acc = train_accuracy.result().numpy()
train_recall.update_state(y_train, model.predict(X_train))
train_precision.update_state(y_train, model.predict(X_train))

print("Accuracy on train set:", train_acc)
print("Recall on train set:", train_recall.result().numpy())
print("Precision on train set:", train_precision.result().numpy())


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 405ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 385ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 381ms/step
Accuracy on train set: 1.0
Recall on train set: 1.0
Precision on train set: 1.0


In [33]:
test_accuracy = tf.keras.metrics.Accuracy()
test_recall = tf.keras.metrics.Recall()
test_precision = tf.keras.metrics.Precision()

test_accuracy.update_state(tf.argmax(y_test, axis=1), tf.argmax(model.predict(X_test), axis=1))
test_acc = test_accuracy.result().numpy()
test_recall.update_state(y_test, model.predict(X_test))
test_precision.update_state(y_test, model.predict(X_test))

print("Recall on test set:", test_recall.result().numpy())
print("Precision on test set:", test_precision.result().numpy())
print("Accuracy on test set:", test_acc)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step 
Recall on test set: 0.90909094
Precision on test set: 0.90909094
Accuracy on test set: 0.90909094
